In [1]:
user = "KELLIE"

if user == "KELLIE":
    dir_main = "/Users/Kellie/Documents/data301/proj/"
elif user == "MUGEN":
    dir_main = ""
else:
    raise Error
    
data_dir = dir_main + "datasets/"

In [ ]:
!ls $data_dir

## TODO:
- won't clean urls
- delete duplicate ids within a single month

- average_pledge_per_person
- 

# Cleaning

In [2]:
import pandas as pd
import numpy as np
import os
import json
from pandas.io.json import json_normalize

pd.options.display.max_rows = 10

In [ ]:
month_df = pd.DataFrame()
folder_name = "Kickstarter_2019-01-17/"
dir_contents = os.listdir(data_dir + folder_name)

for ks in dir_contents:
    baby_df = (pd.read_csv(data_dir +
                           folder_name +
                           ks)[["backers_count", "blurb", "category", "country", "created_at",
                                "currency", "deadline", "goal", "id", "launched_at", "name",
                                "pledged", "slug", "spotlight", "staff_pick", "state",
                                "state_changed_at", "urls", "usd_pledged"]])
    month_df = pd.concat([month_df, baby_df], ignore_index=True)

month_df.head()

In [ ]:
month_df.shape

## Clean JSON Variables

In [ ]:
import ast

json_cols = ["category", "urls"]
#json_cols = ["category", "creator", "photo", "urls", "location"]
# TODO: weren't able to successfully convert creator and location to JSON.
# ignoring these cols and moving on for now, but should revisit later

for col in json_cols:
  print(col)
  month_df[col] = month_df[col].fillna('')
  month_df[col] = month_df[col].apply(ast.literal_eval)
  
month_df.head(3)

In [ ]:
category_exp = json_normalize(month_df["category"])
month_df = month_df.drop("category", axis=1)
month_df.head()

In [ ]:
category_tups = category_exp.slug.str.split("/").apply(pd.Series)
category_tups.columns = ["parent_category", "category"]
month_df = pd.concat([month_df, category_tups], axis=1)
month_df.head()

In [ ]:
urls_exp = json_normalize(month_df["urls"])
urls_exp = urls_exp[["web.project", "web.rewards"]]
urls_exp.columns = ["project_url", "rewards_url"]
urls_exp.head()

In [ ]:
month_df = pd.concat([month_df, urls_exp], axis=1)
month_df = month_df.drop("urls", axis=1)
month_df.head()

## Clean Timestamps

In [ ]:
from datetime import datetime

timestamps = ["created_at", "state_changed_at", "launched_at", "deadline"]
for ts in timestamps:
    month_df[ts + "_clean"] = month_df[ts].apply(datetime.utcfromtimestamp)
month_df.head()

In [ ]:
month_df["project_duration"] = month_df["deadline"] - month_df["created_at"]
month_df["time_til_state_changed"] = (month_df["state_changed_at"] -
                                      month_df["created_at"])
month_df.head()

In [ ]:
month_df.to_csv(data_dir + "processed/" + "201901.csv")

## Process every month from Jan 2016 to Feb 2019

In [ ]:
def collect_month_df(date):
    month_df = pd.DataFrame()
    folder_name = "Kickstarter_" + date + "/"

    for ks in dir_contents:
        baby_df = (pd.read_csv(data_dir +
                               folder_name +
                               ks)[["backers_count", "blurb", "category", "country", "created_at",
                                    "currency", "deadline", "goal", "id", "launched_at", "name",
                                    "pledged", "slug", "spotlight", "staff_pick", "state",
                                    "state_changed_at", "urls", "usd_pledged"]])
        month_df = pd.concat([month_df, baby_df], ignore_index=True)

    return month_df

In [76]:
def convert_strings_to_dict(month_df, cols=["category", "urls"]):
    for col in cols:
        month_df[col] = month_df[col].fillna('')
        month_df[col] = month_df[col].apply(ast.literal_eval)
    return month_df

In [56]:
import ast

def clean_json(month_df, json_cols = ["category", "urls"]):
#     for col in json_cols:
#         print(col)
#         month_df[col] = month_df[col].fillna('')
#         month_df[col] = month_df[col].apply(ast.literal_eval)
    month_df = convert_strings_to_dict(month_df)

    category_exp = json_normalize(month_df["category"])
    category_tups = category_exp.slug.str.split("/").apply(pd.Series)
    category_tups.columns = ["parent_category", "category"]
    month_df = month_df.drop("category", axis=1)
    month_df = pd.concat([month_df, category_tups], axis=1)

    urls_exp = json_normalize(month_df["urls"])
    urls_exp = urls_exp[["web.project", "web.rewards"]]
    urls_exp.columns = ["project_url", "rewards_url"]
    month_df = month_df.drop("urls", axis=1)
    month_df = pd.concat([month_df, urls_exp], axis=1)
    
    return month_df

In [71]:
def clean_category(month_df):
    category_exp = json_normalize(month_df["category"])
    category_tups = category_exp.slug.str.split("/").apply(pd.Series)
    category_tups.columns = ["parent_category", "category"]
    month_df = month_df.drop("category", axis=1)
    month_df = pd.concat([month_df, category_tups], axis=1)
    return month_df

def clean_urls(month_df):
    urls_exp = json_normalize(month_df["urls"])
    urls_exp = urls_exp[["web.project", "web.rewards"]]
    urls_exp.columns = ["project_url", "rewards_url"]
    month_df = month_df.drop("urls", axis=1)
    month_df = pd.concat([month_df, urls_exp], axis=1)
    return month_df

In [89]:
def clean_json(month_df):
    month_df = convert_strings_to_dict(month_df)
    month_df = clean_urls(month_df)
    month_df = clean_category(month_df)
    return month_df

In [4]:
from datetime import datetime

def clean_timestamps(month_df,
                     time_cols = ["created_at", "state_changed_at",
                                  "launched_at", "deadline"]):
    for ts in time_cols:
        month_df[ts + "_clean"] = month_df[ts].apply(datetime.utcfromtimestamp)
    month_df["project_duration"] = (month_df["deadline"] -
                                    month_df["created_at"])
    month_df["time_til_state_changed"] = (month_df["state_changed_at"] -
                                          month_df["created_at"])
    return month_df

In [ ]:
def create_df_for_month(month):
    month_df = collect_month_df(month)
    month_df = clean_json(month_df)
    month_df = clean_timestamps(month_df)
    return month_df

In [ ]:
month_long = "2019-01-17"
month_abbv = "201901"

ks = create_df_for_month(month_long)
print(ks.shape)
ks.to_csv(data_dir + "processed/" + month_abbv + ".csv")

In [109]:
date_str = "2019-01-17"
month_folder = "Kickstarter_" + date_str + "/"
file_num = 26
path = data_dir + month_folder + "Kickstarter0" + str(file_num) + ".csv"
month_df = pd.read_csv(path)[["backers_count", "blurb", "category", "country", "created_at",
                              "currency", "deadline", "goal", "id", "launched_at", "name",
                              "pledged", "slug", "spotlight", "staff_pick", "state",
                              "state_changed_at", "urls", "usd_pledged"]]
month_df.drop_duplicates(inplace=True)
month_df.head()

,backers_count,blurb,category,country,created_at,currency,deadline,goal,id,launched_at,name,pledged,slug,spotlight,staff_pick,state,state_changed_at,urls,usd_pledged
0,25,"WARHEAD is a darkly surreal story about Adam, ...","{""id"":252,""name"":""Graphic Novels"",""slug"":""comi...",US,1493677118,USD,1495177140,1250.0,2052271812,1493854981,WARHEAD: Vol. 1 & Issue 6,1335.0,warhead-vol-1-and-issue-6,True,False,successful,1495177140,"{""web"":{""project"":""https://www.kickstarter.com...",1335.000000
1,17,So much work and many joyful memories have gon...,"{""id"":256,""name"":""Spaces"",""slug"":""dance/spaces...",US,1452626174,USD,1455220800,10000.0,494217614,1452718881,Save Rebel Circus Arts (Canceled),1007.0,save-rebel-circus-arts,False,False,canceled,1453777746,"{""web"":{""project"":""https://www.kickstarter.com...",1007.000000
2,942,1989 Golden Joystick Award-winning hero Rocket...,"{""id"":35,""name"":""Video Games"",""slug"":""games/vi...",US,1413271854,USD,1418860898,89999.0,1002664274,1415836898,Rocket Ranger Reloaded,90715.0,rocket-ranger-reloaded,True,False,successful,1418860899,"{""web"":{""project"":""https://www.kickstarter.com...",90715.000000
3,17,The secret is out - Geeks Do It Better! You kn...,"{""id"":263,""name"":""Apparel"",""slug"":""fashion/app...",AU,1428750049,AUD,1430234561,600.0,1995902400,1428852161,"""Geeks Do It Better"" t-shirt by Silver Fox Comics",901.0,geeks-do-it-better-by-silver-fox-comics-sydney...,True,False,successful,1430234564,"{""web"":{""project"":""https://www.kickstarter.com...",691.910255
4,21,Having just recorded our second single -'LOVEB...,"{""id"":40,""name"":""Indie Rock"",""slug"":""music/ind...",GB,1436726004,GBP,1439327241,170.0,172088720,1436735241,SURGE - 'LOVEBLOOD' 7'' Coloured Vinyl Single ...,285.0,surge-loveblood-7-coloured-vinyl-single-release,True,False,successful,1439327241,"{""web"":{""project"":""https://www.kickstarter.com...",442.303453


In [110]:
month_df = convert_strings_to_dict(month_df)
month_df.head()

,backers_count,blurb,category,country,created_at,currency,deadline,goal,id,launched_at,name,pledged,slug,spotlight,staff_pick,state,state_changed_at,urls,usd_pledged
0,25,"WARHEAD is a darkly surreal story about Adam, ...","{'id': 252, 'name': 'Graphic Novels', 'slug': ...",US,1493677118,USD,1495177140,1250.0,2052271812,1493854981,WARHEAD: Vol. 1 & Issue 6,1335.0,warhead-vol-1-and-issue-6,True,False,successful,1495177140,{'web': {'project': 'https://www.kickstarter.c...,1335.000000
1,17,So much work and many joyful memories have gon...,"{'id': 256, 'name': 'Spaces', 'slug': 'dance/s...",US,1452626174,USD,1455220800,10000.0,494217614,1452718881,Save Rebel Circus Arts (Canceled),1007.0,save-rebel-circus-arts,False,False,canceled,1453777746,{'web': {'project': 'https://www.kickstarter.c...,1007.000000
2,942,1989 Golden Joystick Award-winning hero Rocket...,"{'id': 35, 'name': 'Video Games', 'slug': 'gam...",US,1413271854,USD,1418860898,89999.0,1002664274,1415836898,Rocket Ranger Reloaded,90715.0,rocket-ranger-reloaded,True,False,successful,1418860899,{'web': {'project': 'https://www.kickstarter.c...,90715.000000
3,17,The secret is out - Geeks Do It Better! You kn...,"{'id': 263, 'name': 'Apparel', 'slug': 'fashio...",AU,1428750049,AUD,1430234561,600.0,1995902400,1428852161,"""Geeks Do It Better"" t-shirt by Silver Fox Comics",901.0,geeks-do-it-better-by-silver-fox-comics-sydney...,True,False,successful,1430234564,{'web': {'project': 'https://www.kickstarter.c...,691.910255
4,21,Having just recorded our second single -'LOVEB...,"{'id': 40, 'name': 'Indie Rock', 'slug': 'musi...",GB,1436726004,GBP,1439327241,170.0,172088720,1436735241,SURGE - 'LOVEBLOOD' 7'' Coloured Vinyl Single ...,285.0,surge-loveblood-7-coloured-vinyl-single-release,True,False,successful,1439327241,{'web': {'project': 'https://www.kickstarter.c...,442.303453


In [111]:
clean_category(month_df)
clean_urls(month_df)

,backers_count,blurb,category,country,created_at,currency,deadline,goal,id,launched_at,name,pledged,slug,spotlight,staff_pick,state,state_changed_at,usd_pledged,project_url,rewards_url
0,25.0,"WARHEAD is a darkly surreal story about Adam, ...","{'id': 252, 'name': 'Graphic Novels', 'slug': ...",US,1.493677e+09,USD,1.495177e+09,1250.0,2.052272e+09,1.493855e+09,WARHEAD: Vol. 1 & Issue 6,1335.00,warhead-vol-1-and-issue-6,True,False,successful,1.495177e+09,1335.000000,https://www.kickstarter.com/projects/203825093...,https://www.kickstarter.com/projects/203825093...
1,17.0,So much work and many joyful memories have gon...,"{'id': 256, 'name': 'Spaces', 'slug': 'dance/s...",US,1.452626e+09,USD,1.455221e+09,10000.0,4.942176e+08,1.452719e+09,Save Rebel Circus Arts (Canceled),1007.00,save-rebel-circus-arts,False,False,canceled,1.453778e+09,1007.000000,https://www.kickstarter.com/projects/91318819/...,https://www.kickstarter.com/projects/91318819/...
2,942.0,1989 Golden Joystick Award-winning hero Rocket...,"{'id': 35, 'name': 'Video Games', 'slug': 'gam...",US,1.413272e+09,USD,1.418861e+09,89999.0,1.002664e+09,1.415837e+09,Rocket Ranger Reloaded,90715.00,rocket-ranger-reloaded,True,False,successful,1.418861e+09,90715.000000,https://www.kickstarter.com/projects/850516062...,https://www.kickstarter.com/projects/850516062...
3,17.0,The secret is out - Geeks Do It Better! You kn...,"{'id': 263, 'name': 'Apparel', 'slug': 'fashio...",AU,1.428750e+09,AUD,1.430235e+09,600.0,1.995902e+09,1.428852e+09,"""Geeks Do It Better"" t-shirt by Silver Fox Comics",901.00,geeks-do-it-better-by-silver-fox-comics-sydney...,True,False,successful,1.430235e+09,691.910255,https://www.kickstarter.com/projects/silverfox...,https://www.kickstarter.com/projects/silverfox...
4,21.0,Having just recorded our second single -'LOVEB...,"{'id': 40, 'name': 'Indie Rock', 'slug': 'musi...",GB,1.436726e+09,GBP,1.439327e+09,170.0,1.720887e+08,1.436735e+09,SURGE - 'LOVEBLOOD' 7'' Coloured Vinyl Single ...,285.00,surge-loveblood-7-coloured-vinyl-single-release,True,False,successful,1.439327e+09,442.303453,https://www.kickstarter.com/projects/204644917...,https://www.kickstarter.com/projects/204644917...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3820,45.0,A film about a 16 year old girl's response to ...,"{'id': 31, 'name': 'Narrative Film', 'slug': '...",US,1.285194e+09,USD,1.288655e+09,2000.0,1.126519e+09,1.285199e+09,"""Dollhouse in the Desert""- a thesis film about...",2134.00,dollhouse-in-the-desert-a-thesis-film-about-a-...,True,False,successful,1.288655e+09,2134.000000,NaN,NaN
3821,52.0,WAXWING is an exciting new world premiere of m...,"{'id': 285, 'name': 'Plays', 'slug': 'theater/...",US,1.323125e+09,USD,1.326928e+09,1500.0,1.283694e+09,1.323222e+09,Waxwing: A New Play,2002.22,waxwing-a-new-play,True,True,successful,1.326928e+09,2002.220000,NaN,NaN
3822,1.0,"A completely green, self sustaining Commercial...","{'id': 309, 'name': 'Farms', 'slug': 'food/far...",US,1.417397e+09,USD,1.421292e+09,150000.0,2.020280e+08,1.418700e+09,promise land farms,15.00,promise-land-farms,False,False,failed,1.421292e+09,15.000000,NaN,NaN
3823,0.0,"inspired by the 80s, 90s synthpop, electronica...","{'id': 38, 'name': 'Electronic Music', 'slug':...",CA,1.406244e+09,CAD,1.409199e+09,12000.0,1.663043e+09,1.406607e+09,Machine Mother,0.00,machine-mother,False,False,failed,1.409199e+09,0.000000,NaN,NaN


In [112]:
month_df = clean_category(month_df)
clean_urls(month_df)

AttributeError: 'float' object has no attribute 'values'

In [116]:
json.loads(month_df.urls[0])

TypeError: the JSON object must be str, bytes or bytearray, not dict

In [102]:
clean_urls(month_df)

,backers_count,blurb,category,country,created_at,currency,deadline,goal,id,launched_at,name,pledged,slug,spotlight,staff_pick,state,state_changed_at,usd_pledged,project_url,rewards_url
0,25.0,"WARHEAD is a darkly surreal story about Adam, ...","{'id': 252, 'name': 'Graphic Novels', 'slug': ...",US,1.493677e+09,USD,1.495177e+09,1250.0,2.052272e+09,1.493855e+09,WARHEAD: Vol. 1 & Issue 6,1335.00,warhead-vol-1-and-issue-6,True,False,successful,1.495177e+09,1335.000000,https://www.kickstarter.com/projects/203825093...,https://www.kickstarter.com/projects/203825093...
1,17.0,So much work and many joyful memories have gon...,"{'id': 256, 'name': 'Spaces', 'slug': 'dance/s...",US,1.452626e+09,USD,1.455221e+09,10000.0,4.942176e+08,1.452719e+09,Save Rebel Circus Arts (Canceled),1007.00,save-rebel-circus-arts,False,False,canceled,1.453778e+09,1007.000000,https://www.kickstarter.com/projects/91318819/...,https://www.kickstarter.com/projects/91318819/...
2,942.0,1989 Golden Joystick Award-winning hero Rocket...,"{'id': 35, 'name': 'Video Games', 'slug': 'gam...",US,1.413272e+09,USD,1.418861e+09,89999.0,1.002664e+09,1.415837e+09,Rocket Ranger Reloaded,90715.00,rocket-ranger-reloaded,True,False,successful,1.418861e+09,90715.000000,https://www.kickstarter.com/projects/850516062...,https://www.kickstarter.com/projects/850516062...
3,17.0,The secret is out - Geeks Do It Better! You kn...,"{'id': 263, 'name': 'Apparel', 'slug': 'fashio...",AU,1.428750e+09,AUD,1.430235e+09,600.0,1.995902e+09,1.428852e+09,"""Geeks Do It Better"" t-shirt by Silver Fox Comics",901.00,geeks-do-it-better-by-silver-fox-comics-sydney...,True,False,successful,1.430235e+09,691.910255,https://www.kickstarter.com/projects/silverfox...,https://www.kickstarter.com/projects/silverfox...
4,21.0,Having just recorded our second single -'LOVEB...,"{'id': 40, 'name': 'Indie Rock', 'slug': 'musi...",GB,1.436726e+09,GBP,1.439327e+09,170.0,1.720887e+08,1.436735e+09,SURGE - 'LOVEBLOOD' 7'' Coloured Vinyl Single ...,285.00,surge-loveblood-7-coloured-vinyl-single-release,True,False,successful,1.439327e+09,442.303453,https://www.kickstarter.com/projects/204644917...,https://www.kickstarter.com/projects/204644917...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3820,45.0,A film about a 16 year old girl's response to ...,"{'id': 31, 'name': 'Narrative Film', 'slug': '...",US,1.285194e+09,USD,1.288655e+09,2000.0,1.126519e+09,1.285199e+09,"""Dollhouse in the Desert""- a thesis film about...",2134.00,dollhouse-in-the-desert-a-thesis-film-about-a-...,True,False,successful,1.288655e+09,2134.000000,NaN,NaN
3821,52.0,WAXWING is an exciting new world premiere of m...,"{'id': 285, 'name': 'Plays', 'slug': 'theater/...",US,1.323125e+09,USD,1.326928e+09,1500.0,1.283694e+09,1.323222e+09,Waxwing: A New Play,2002.22,waxwing-a-new-play,True,True,successful,1.326928e+09,2002.220000,NaN,NaN
3822,1.0,"A completely green, self sustaining Commercial...","{'id': 309, 'name': 'Farms', 'slug': 'food/far...",US,1.417397e+09,USD,1.421292e+09,150000.0,2.020280e+08,1.418700e+09,promise land farms,15.00,promise-land-farms,False,False,failed,1.421292e+09,15.000000,NaN,NaN
3823,0.0,"inspired by the 80s, 90s synthpop, electronica...","{'id': 38, 'name': 'Electronic Music', 'slug':...",CA,1.406244e+09,CAD,1.409199e+09,12000.0,1.663043e+09,1.406607e+09,Machine Mother,0.00,machine-mother,False,False,failed,1.409199e+09,0.000000,NaN,NaN


In [105]:
month_df = convert_strings_to_dict(month_df)
month_df = clean_category(month_df)
month_df.head()

SyntaxError: unexpected EOF while parsing (<unknown>, line 1)

In [97]:
month_df = clean_urls(month_df)
month_df.head()

AttributeError: 'float' object has no attribute 'values'

In [78]:
month_df = convert_strings_to_dict(month_df)
month_df = clean_category(month_df)
month_df.head()

,backers_count,blurb,country,created_at,currency,deadline,goal,id,launched_at,name,pledged,slug,spotlight,staff_pick,state,state_changed_at,urls,usd_pledged,parent_category,category
0,25.0,"WARHEAD is a darkly surreal story about Adam, ...",US,1.493677e+09,USD,1.495177e+09,1250.0,2.052272e+09,1.493855e+09,WARHEAD: Vol. 1 & Issue 6,1335.0,warhead-vol-1-and-issue-6,True,False,successful,1.495177e+09,{'web': {'project': 'https://www.kickstarter.c...,1335.000000,comics,graphic novels
1,17.0,So much work and many joyful memories have gon...,US,1.452626e+09,USD,1.455221e+09,10000.0,4.942176e+08,1.452719e+09,Save Rebel Circus Arts (Canceled),1007.0,save-rebel-circus-arts,False,False,canceled,1.453778e+09,{'web': {'project': 'https://www.kickstarter.c...,1007.000000,dance,spaces
2,942.0,1989 Golden Joystick Award-winning hero Rocket...,US,1.413272e+09,USD,1.418861e+09,89999.0,1.002664e+09,1.415837e+09,Rocket Ranger Reloaded,90715.0,rocket-ranger-reloaded,True,False,successful,1.418861e+09,{'web': {'project': 'https://www.kickstarter.c...,90715.000000,games,video games
3,17.0,The secret is out - Geeks Do It Better! You kn...,AU,1.428750e+09,AUD,1.430235e+09,600.0,1.995902e+09,1.428852e+09,"""Geeks Do It Better"" t-shirt by Silver Fox Comics",901.0,geeks-do-it-better-by-silver-fox-comics-sydney...,True,False,successful,1.430235e+09,{'web': {'project': 'https://www.kickstarter.c...,691.910255,fashion,apparel
4,21.0,Having just recorded our second single -'LOVEB...,GB,1.436726e+09,GBP,1.439327e+09,170.0,1.720887e+08,1.436735e+09,SURGE - 'LOVEBLOOD' 7'' Coloured Vinyl Single ...,285.0,surge-loveblood-7-coloured-vinyl-single-release,True,False,successful,1.439327e+09,{'web': {'project': 'https://www.kickstarter.c...,442.303453,music,indie rock


In [85]:
month_df = convert_strings_to_dict(month_df)
month_df.head()

,backers_count,blurb,category,country,created_at,currency,deadline,goal,id,launched_at,name,pledged,slug,spotlight,staff_pick,state,state_changed_at,urls,usd_pledged
0,25,"WARHEAD is a darkly surreal story about Adam, ...","{'id': 252, 'name': 'Graphic Novels', 'slug': ...",US,1493677118,USD,1495177140,1250.0,2052271812,1493854981,WARHEAD: Vol. 1 & Issue 6,1335.0,warhead-vol-1-and-issue-6,True,False,successful,1495177140,{'web': {'project': 'https://www.kickstarter.c...,1335.000000
1,17,So much work and many joyful memories have gon...,"{'id': 256, 'name': 'Spaces', 'slug': 'dance/s...",US,1452626174,USD,1455220800,10000.0,494217614,1452718881,Save Rebel Circus Arts (Canceled),1007.0,save-rebel-circus-arts,False,False,canceled,1453777746,{'web': {'project': 'https://www.kickstarter.c...,1007.000000
2,942,1989 Golden Joystick Award-winning hero Rocket...,"{'id': 35, 'name': 'Video Games', 'slug': 'gam...",US,1413271854,USD,1418860898,89999.0,1002664274,1415836898,Rocket Ranger Reloaded,90715.0,rocket-ranger-reloaded,True,False,successful,1418860899,{'web': {'project': 'https://www.kickstarter.c...,90715.000000
3,17,The secret is out - Geeks Do It Better! You kn...,"{'id': 263, 'name': 'Apparel', 'slug': 'fashio...",AU,1428750049,AUD,1430234561,600.0,1995902400,1428852161,"""Geeks Do It Better"" t-shirt by Silver Fox Comics",901.0,geeks-do-it-better-by-silver-fox-comics-sydney...,True,False,successful,1430234564,{'web': {'project': 'https://www.kickstarter.c...,691.910255
4,21,Having just recorded our second single -'LOVEB...,"{'id': 40, 'name': 'Indie Rock', 'slug': 'musi...",GB,1436726004,GBP,1439327241,170.0,172088720,1436735241,SURGE - 'LOVEBLOOD' 7'' Coloured Vinyl Single ...,285.0,surge-loveblood-7-coloured-vinyl-single-release,True,False,successful,1439327241,{'web': {'project': 'https://www.kickstarter.c...,442.303453


In [87]:
clean_urls(month_df)

,backers_count,blurb,category,country,created_at,currency,deadline,goal,id,launched_at,name,pledged,slug,spotlight,staff_pick,state,state_changed_at,usd_pledged,project_url,rewards_url
0,25.0,"WARHEAD is a darkly surreal story about Adam, ...","{'id': 252, 'name': 'Graphic Novels', 'slug': ...",US,1.493677e+09,USD,1.495177e+09,1250.0,2.052272e+09,1.493855e+09,WARHEAD: Vol. 1 & Issue 6,1335.00,warhead-vol-1-and-issue-6,True,False,successful,1.495177e+09,1335.000000,https://www.kickstarter.com/projects/203825093...,https://www.kickstarter.com/projects/203825093...
1,17.0,So much work and many joyful memories have gon...,"{'id': 256, 'name': 'Spaces', 'slug': 'dance/s...",US,1.452626e+09,USD,1.455221e+09,10000.0,4.942176e+08,1.452719e+09,Save Rebel Circus Arts (Canceled),1007.00,save-rebel-circus-arts,False,False,canceled,1.453778e+09,1007.000000,https://www.kickstarter.com/projects/91318819/...,https://www.kickstarter.com/projects/91318819/...
2,942.0,1989 Golden Joystick Award-winning hero Rocket...,"{'id': 35, 'name': 'Video Games', 'slug': 'gam...",US,1.413272e+09,USD,1.418861e+09,89999.0,1.002664e+09,1.415837e+09,Rocket Ranger Reloaded,90715.00,rocket-ranger-reloaded,True,False,successful,1.418861e+09,90715.000000,https://www.kickstarter.com/projects/850516062...,https://www.kickstarter.com/projects/850516062...
3,17.0,The secret is out - Geeks Do It Better! You kn...,"{'id': 263, 'name': 'Apparel', 'slug': 'fashio...",AU,1.428750e+09,AUD,1.430235e+09,600.0,1.995902e+09,1.428852e+09,"""Geeks Do It Better"" t-shirt by Silver Fox Comics",901.00,geeks-do-it-better-by-silver-fox-comics-sydney...,True,False,successful,1.430235e+09,691.910255,https://www.kickstarter.com/projects/silverfox...,https://www.kickstarter.com/projects/silverfox...
4,21.0,Having just recorded our second single -'LOVEB...,"{'id': 40, 'name': 'Indie Rock', 'slug': 'musi...",GB,1.436726e+09,GBP,1.439327e+09,170.0,1.720887e+08,1.436735e+09,SURGE - 'LOVEBLOOD' 7'' Coloured Vinyl Single ...,285.00,surge-loveblood-7-coloured-vinyl-single-release,True,False,successful,1.439327e+09,442.303453,https://www.kickstarter.com/projects/204644917...,https://www.kickstarter.com/projects/204644917...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3820,45.0,A film about a 16 year old girl's response to ...,"{'id': 31, 'name': 'Narrative Film', 'slug': '...",US,1.285194e+09,USD,1.288655e+09,2000.0,1.126519e+09,1.285199e+09,"""Dollhouse in the Desert""- a thesis film about...",2134.00,dollhouse-in-the-desert-a-thesis-film-about-a-...,True,False,successful,1.288655e+09,2134.000000,NaN,NaN
3821,52.0,WAXWING is an exciting new world premiere of m...,"{'id': 285, 'name': 'Plays', 'slug': 'theater/...",US,1.323125e+09,USD,1.326928e+09,1500.0,1.283694e+09,1.323222e+09,Waxwing: A New Play,2002.22,waxwing-a-new-play,True,True,successful,1.326928e+09,2002.220000,NaN,NaN
3822,1.0,"A completely green, self sustaining Commercial...","{'id': 309, 'name': 'Farms', 'slug': 'food/far...",US,1.417397e+09,USD,1.421292e+09,150000.0,2.020280e+08,1.418700e+09,promise land farms,15.00,promise-land-farms,False,False,failed,1.421292e+09,15.000000,NaN,NaN
3823,0.0,"inspired by the 80s, 90s synthpop, electronica...","{'id': 38, 'name': 'Electronic Music', 'slug':...",CA,1.406244e+09,CAD,1.409199e+09,12000.0,1.663043e+09,1.406607e+09,Machine Mother,0.00,machine-mother,False,False,failed,1.409199e+09,0.000000,NaN,NaN


In [88]:
clean_category(month_df)
month_df.head()

,backers_count,blurb,category,country,created_at,currency,deadline,goal,id,launched_at,name,pledged,slug,spotlight,staff_pick,state,state_changed_at,urls,usd_pledged
0,25,"WARHEAD is a darkly surreal story about Adam, ...","{'id': 252, 'name': 'Graphic Novels', 'slug': ...",US,1493677118,USD,1495177140,1250.0,2052271812,1493854981,WARHEAD: Vol. 1 & Issue 6,1335.0,warhead-vol-1-and-issue-6,True,False,successful,1495177140,{'web': {'project': 'https://www.kickstarter.c...,1335.000000
1,17,So much work and many joyful memories have gon...,"{'id': 256, 'name': 'Spaces', 'slug': 'dance/s...",US,1452626174,USD,1455220800,10000.0,494217614,1452718881,Save Rebel Circus Arts (Canceled),1007.0,save-rebel-circus-arts,False,False,canceled,1453777746,{'web': {'project': 'https://www.kickstarter.c...,1007.000000
2,942,1989 Golden Joystick Award-winning hero Rocket...,"{'id': 35, 'name': 'Video Games', 'slug': 'gam...",US,1413271854,USD,1418860898,89999.0,1002664274,1415836898,Rocket Ranger Reloaded,90715.0,rocket-ranger-reloaded,True,False,successful,1418860899,{'web': {'project': 'https://www.kickstarter.c...,90715.000000
3,17,The secret is out - Geeks Do It Better! You kn...,"{'id': 263, 'name': 'Apparel', 'slug': 'fashio...",AU,1428750049,AUD,1430234561,600.0,1995902400,1428852161,"""Geeks Do It Better"" t-shirt by Silver Fox Comics",901.0,geeks-do-it-better-by-silver-fox-comics-sydney...,True,False,successful,1430234564,{'web': {'project': 'https://www.kickstarter.c...,691.910255
4,21,Having just recorded our second single -'LOVEB...,"{'id': 40, 'name': 'Indie Rock', 'slug': 'musi...",GB,1436726004,GBP,1439327241,170.0,172088720,1436735241,SURGE - 'LOVEBLOOD' 7'' Coloured Vinyl Single ...,285.0,surge-loveblood-7-coloured-vinyl-single-release,True,False,successful,1439327241,{'web': {'project': 'https://www.kickstarter.c...,442.303453


In [80]:
urls_exp = json_normalize(month_df["urls"])
urls_exp = urls_exp[["web.project", "web.rewards"]]
urls_exp.columns = ["project_url", "rewards_url"]
month_df = month_df.drop("urls", axis=1)
month_df = pd.concat([month_df, urls_exp], axis=1)

AttributeError: 'float' object has no attribute 'values'

In [67]:
month_df = convert_strings_to_dict(month_df)
month_df.head()

category
urls


,backers_count,blurb,category,country,created_at,currency,deadline,goal,id,launched_at,name,pledged,slug,spotlight,staff_pick,state,state_changed_at,urls,usd_pledged
0,25,"WARHEAD is a darkly surreal story about Adam, ...","{'id': 252, 'name': 'Graphic Novels', 'slug': ...",US,1493677118,USD,1495177140,1250.0,2052271812,1493854981,WARHEAD: Vol. 1 & Issue 6,1335.0,warhead-vol-1-and-issue-6,True,False,successful,1495177140,{'web': {'project': 'https://www.kickstarter.c...,1335.000000
1,17,So much work and many joyful memories have gon...,"{'id': 256, 'name': 'Spaces', 'slug': 'dance/s...",US,1452626174,USD,1455220800,10000.0,494217614,1452718881,Save Rebel Circus Arts (Canceled),1007.0,save-rebel-circus-arts,False,False,canceled,1453777746,{'web': {'project': 'https://www.kickstarter.c...,1007.000000
2,942,1989 Golden Joystick Award-winning hero Rocket...,"{'id': 35, 'name': 'Video Games', 'slug': 'gam...",US,1413271854,USD,1418860898,89999.0,1002664274,1415836898,Rocket Ranger Reloaded,90715.0,rocket-ranger-reloaded,True,False,successful,1418860899,{'web': {'project': 'https://www.kickstarter.c...,90715.000000
3,17,The secret is out - Geeks Do It Better! You kn...,"{'id': 263, 'name': 'Apparel', 'slug': 'fashio...",AU,1428750049,AUD,1430234561,600.0,1995902400,1428852161,"""Geeks Do It Better"" t-shirt by Silver Fox Comics",901.0,geeks-do-it-better-by-silver-fox-comics-sydney...,True,False,successful,1430234564,{'web': {'project': 'https://www.kickstarter.c...,691.910255
4,21,Having just recorded our second single -'LOVEB...,"{'id': 40, 'name': 'Indie Rock', 'slug': 'musi...",GB,1436726004,GBP,1439327241,170.0,172088720,1436735241,SURGE - 'LOVEBLOOD' 7'' Coloured Vinyl Single ...,285.0,surge-loveblood-7-coloured-vinyl-single-release,True,False,successful,1439327241,{'web': {'project': 'https://www.kickstarter.c...,442.303453


In [70]:
month_df = clean_json(month_df)
month_df = clean_timestamps(month_df)
month_df.head()

category
urls


AttributeError: 'float' object has no attribute 'values'

In [46]:
month_df.head()

,backers_count,blurb,category,country,created_at,currency,deadline,goal,id,launched_at,name,pledged,slug,spotlight,staff_pick,state,state_changed_at,urls,usd_pledged
0,25,"WARHEAD is a darkly surreal story about Adam, ...","{'id': 252, 'name': 'Graphic Novels', 'slug': ...",US,1493677118,USD,1495177140,1250.0,2052271812,1493854981,WARHEAD: Vol. 1 & Issue 6,1335.0,warhead-vol-1-and-issue-6,True,False,successful,1495177140,{'web': {'project': 'https://www.kickstarter.c...,1335.000000
1,17,So much work and many joyful memories have gon...,"{'id': 256, 'name': 'Spaces', 'slug': 'dance/s...",US,1452626174,USD,1455220800,10000.0,494217614,1452718881,Save Rebel Circus Arts (Canceled),1007.0,save-rebel-circus-arts,False,False,canceled,1453777746,{'web': {'project': 'https://www.kickstarter.c...,1007.000000
2,942,1989 Golden Joystick Award-winning hero Rocket...,"{'id': 35, 'name': 'Video Games', 'slug': 'gam...",US,1413271854,USD,1418860898,89999.0,1002664274,1415836898,Rocket Ranger Reloaded,90715.0,rocket-ranger-reloaded,True,False,successful,1418860899,{'web': {'project': 'https://www.kickstarter.c...,90715.000000
3,17,The secret is out - Geeks Do It Better! You kn...,"{'id': 263, 'name': 'Apparel', 'slug': 'fashio...",AU,1428750049,AUD,1430234561,600.0,1995902400,1428852161,"""Geeks Do It Better"" t-shirt by Silver Fox Comics",901.0,geeks-do-it-better-by-silver-fox-comics-sydney...,True,False,successful,1430234564,{'web': {'project': 'https://www.kickstarter.c...,691.910255
4,21,Having just recorded our second single -'LOVEB...,"{'id': 40, 'name': 'Indie Rock', 'slug': 'musi...",GB,1436726004,GBP,1439327241,170.0,172088720,1436735241,SURGE - 'LOVEBLOOD' 7'' Coloured Vinyl Single ...,285.0,surge-loveblood-7-coloured-vinyl-single-release,True,False,successful,1439327241,{'web': {'project': 'https://www.kickstarter.c...,442.303453
